In [1]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

# set select statement
sql = """SELECT
B.PVndr_nbr,
B.Mvndr_nbr,
A.ORD_CRT_DT,
A.ORD_QTY,
B.Fcst_crt_wk,
B.FiscalWeek,
B.SOQ_Units,
A.yearplusweekno
FROM `analytics-supplychain-thd.Geraldine.CV_SOQ_HIST_FINAL` B
LEFT JOIN `analytics-supplychain-thd.HES.DOM_FCAST_ACTUAL_QTY` A
  ON B.PVndr_nbr = A.PVNDR_NBR
  AND B.Mvndr_nbr = A.MVNDR_NBR
  AND B.SKU_NBR = cast(A.SKU_NBR AS INT64)
  AND B.yearplusweekno = A.yearplusweekno
WHERE A.ORD_CRT_DT IS NOT NULL
ORDER BY 5; """

# read data from BQ
data = pd.read_gbq(sql, project_id = 'analytics-supplychain-thd', dialect = 'standard')
data

C:\Users\HXS1YSJ\Anaconda3\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Downloading: 100%|█████████████████████████████████████████████████| 134867167/134867167 [05:03<00:00, 444696.68rows/s]


,PVndr_nbr,Mvndr_nbr,ORD_CRT_DT,ORD_QTY,Fcst_crt_wk,FiscalWeek,SOQ_Units,yearplusweekno
0,455575,10612,2020-11-13,672.0,2020-11-02,week02,144.0,202041
1,291923,291923,2020-11-17,8.0,2020-11-02,week03,14.0,202042
2,206820,206820,2020-11-19,5.0,2020-11-02,week03,70.0,202042
3,696008,696006,2020-11-23,240.0,2020-11-02,week04,162.0,202043
4,102180,102148,2020-11-05,2.0,2020-11-02,week01,26.0,202040
...,...,...,...,...,...,...,...,...
134867162,102180,102148,2020-11-25,15.0,2020-10-19,week06,6.0,202043
134867163,696008,696006,2020-11-24,18.0,2020-10-19,week06,162.0,202043
134867164,502378,502378,2020-10-26,5.0,2020-10-19,week02,50.0,202039
134867165,455575,10612,2020-11-06,24.0,2020-10-19,week03,0.0,202040


In [2]:
# calculate percent error and error for each observation
data['Perc_Error'] = ((data['ORD_QTY'] - data['SOQ_Units'])/data['ORD_QTY']) * 100
data['Error'] = data['ORD_QTY'] - data['SOQ_Units']
data

,PVndr_nbr,Mvndr_nbr,ORD_CRT_DT,ORD_QTY,Fcst_crt_wk,FiscalWeek,SOQ_Units,yearplusweekno,Perc_Error,Error
0,455575,10612,2020-11-13,672.0,2020-11-02,week02,144.0,202041,78.571429,528.0
1,291923,291923,2020-11-17,8.0,2020-11-02,week03,14.0,202042,-75.000000,-6.0
2,206820,206820,2020-11-19,5.0,2020-11-02,week03,70.0,202042,-1300.000000,-65.0
3,696008,696006,2020-11-23,240.0,2020-11-02,week04,162.0,202043,32.500000,78.0
4,102180,102148,2020-11-05,2.0,2020-11-02,week01,26.0,202040,-1200.000000,-24.0
...,...,...,...,...,...,...,...,...,...,...
134867162,102180,102148,2020-11-25,15.0,2020-10-19,week06,6.0,202043,60.000000,9.0
134867163,696008,696006,2020-11-24,18.0,2020-10-19,week06,162.0,202043,-800.000000,-144.0
134867164,502378,502378,2020-10-26,5.0,2020-10-19,week02,50.0,202039,-900.000000,-45.0
134867165,455575,10612,2020-11-06,24.0,2020-10-19,week03,0.0,202040,100.000000,24.0


In [3]:
# no function for MAPE in sklearn so definie it here
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [4]:
# save just the FiscalWeek column to a dataframe
FiscalWeek = data['FiscalWeek']

# function to get all unique FiscalWeeks
def get_unique_numbers(FiscalWeek):
    list_of_unique_numbers = []
    unique_numbers = set(FiscalWeek)
    for number in unique_numbers:
        list_of_unique_numbers.append(number)
    return list_of_unique_numbers

# save unique FW to a list
FW_unique = get_unique_numbers(FiscalWeek)

# how many FW do we have with an order quantity?
len(FW_unique)
FW_unique

['week05', 'week07', 'week01', 'week03', 'week06', 'week04', 'week02']

In [5]:
# convert to dataframe
perf_df = pd.DataFrame(columns=['FW','MAPE','MAE','MPE','Mean_Error'])
columns = list(perf_df)

In [6]:
# initialize empty dataframe
df = []

# loop through each FW and calculate performance metrics
for i in FW_unique:
    data_new = data[(data.FiscalWeek == i)]
    FW = i
    MAPE = mean_absolute_percentage_error(data_new['ORD_QTY'],data_new['SOQ_Units'])
    MAE= mean_absolute_error(data_new['ORD_QTY'],data_new['SOQ_Units'])
    MPE = data_new['Perc_Error'].mean()
    Mean_Error = data_new['Error'].mean()
    
    values = [FW,MAPE,MAE,MPE,Mean_Error]
    zipped = zip(columns,values)
    dictionary = dict(zipped)
    
    print(dictionary)
    df.append(dictionary)
    
    perf_df = perf_df.append(df, True)

{'FW': 'week05', 'MAPE': 6072.677118647021, 'MAE': 564.9614138126788, 'MPE': -6039.872482858976, 'Mean_Error': -509.4825882744684}
{'FW': 'week07', 'MAPE': 1104.5404078922293, 'MAE': 141.48252003848438, 'MPE': -1063.260181320794, 'Mean_Error': -68.99697653949812}
{'FW': 'week01', 'MAPE': 7879.41721053365, 'MAE': 742.3048796443619, 'MPE': -7864.054981201674, 'Mean_Error': -714.7576853529644}
{'FW': 'week03', 'MAPE': 6313.628800013512, 'MAE': 552.482765514055, 'MPE': -6271.57567947758, 'Mean_Error': -485.00708505732223}
{'FW': 'week06', 'MAPE': 4457.267881607015, 'MAE': 434.17281354607724, 'MPE': -4419.4698644861355, 'Mean_Error': -371.9399504912552}
{'FW': 'week04', 'MAPE': 6453.574227810765, 'MAE': 580.1745159301773, 'MPE': -6416.626048634601, 'Mean_Error': -520.6434850988257}
{'FW': 'week02', 'MAPE': 6016.581238578148, 'MAE': 524.9362691464504, 'MPE': -5974.717142156711, 'Mean_Error': -458.0817293523438}


In [7]:
# drop any duplicates and sort by FW
perf_df = perf_df.drop_duplicates().sort_values('FW')
perf_df

# in forecasts that are 7 weeks out, we are forecasting on average 69 more units than we end up ordering
# this increases as we get closer to the actual order date
# interesting - less accurate/more error as we get closer to the actual order date

,FW,MAPE,MAE,MPE,Mean_Error
5,week01,7879.417211,742.304880,-7864.054981,-714.757685
27,week02,6016.581239,524.936269,-5974.717142,-458.081729
9,week03,6313.628800,552.482766,-6271.575679,-485.007085
20,week04,6453.574228,580.174516,-6416.626049,-520.643485
0,week05,6072.677119,564.961414,-6039.872483,-509.482588
14,week06,4457.267882,434.172814,-4419.469864,-371.939950
2,week07,1104.540408,141.482520,-1063.260181,-68.996977
